<a href="https://colab.research.google.com/github/simecek/Czech-ULMFiT/blob/master/experiments/CSFD_retrained_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!pip install sentencepiece

In [ ]:
import pandas as pd

from fastai import *
from fastai.text import *

In [ ]:
# GPU test
import torch
#torch.cuda.current_device()
#torch.cuda.device(0)
#torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
!wget http://liks.fav.zcu.cz/sentiment/csfd.zip
!unzip csfd.zip
!ls csfd/*.txt

--2020-07-13 15:48:26--  http://liks.fav.zcu.cz/sentiment/csfd.zip
Resolving liks.fav.zcu.cz (liks.fav.zcu.cz)... 147.228.63.169
Connecting to liks.fav.zcu.cz (liks.fav.zcu.cz)|147.228.63.169|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13734596 (13M) [application/zip]
Saving to: ‘csfd.zip’

csfd.zip            100%[===================>]  13.10M  7.91MB/s    in 1.7s    

2020-07-13 15:48:28 (7.91 MB/s) - ‘csfd.zip’ saved [13734596/13734596]

Archive:  csfd.zip
   creating: csfd/
  inflating: csfd/licence.txt        
  inflating: csfd/negative.txt       
  inflating: csfd/neutral.txt        
  inflating: csfd/positive.txt       
csfd/licence.txt  csfd/negative.txt  csfd/neutral.txt  csfd/positive.txt


In [ ]:
!mkdir models
# you can download cs_wt.pth & cs_wt_vocab.pkl from
# https://drive.google.com/open?id=14b5x5r3x5MeZNZ8Uc4L3ZmiHAiDgKNj2
# https://drive.google.com/open?id=1NZym3XfEWAGJ7L3O56Zk2er6bwjKdJGe
!cp "/content/drive/My Drive/ULMFit-Czech/cs_wt.pth" models/
!cp "/content/drive/My Drive/ULMFit-Czech/cs_wt_vocab.pkl" models/
!mkdir tmp
!wget https://raw.githubusercontent.com/simecek/Czech-ULMFiT/master/language_model/spm.model
!wget https://raw.githubusercontent.com/simecek/Czech-ULMFiT/master/language_model/spm.vocab 
!mv spm* tmp/

--2020-07-13 15:48:38--  https://raw.githubusercontent.com/simecek/Czech-ULMFiT/master/language_model/spm.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 779876 (762K) [application/octet-stream]
Saving to: ‘spm.model’

spm.model           100%[===================>] 761.60K  --.-KB/s    in 0.05s   

2020-07-13 15:48:38 (13.8 MB/s) - ‘spm.model’ saved [779876/779876]

--2020-07-13 15:48:40--  https://raw.githubusercontent.com/simecek/Czech-ULMFiT/master/language_model/spm.vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 568615 (555

In [ ]:
path = Path(".")
pos = (path/'csfd/positive.txt').open().readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,Jako se mě líbila stejně jako jednička. Má sic...,1
1,Sice se rika ze dvojky pokulhavaji za jednicko...,1
2,Tenhle film můžu opravdu kdykoliv. A to nejen ...,1
3,velice povedená krimikomedie s plejádou skvělý...,1
4,Rok 1969. Při výročí invaze střílejí do lidí č...,1


In [ ]:
neg = (path/'csfd/negative.txt').open().readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,Tommy Lee Jonesovi jsou role jako je tato šity...,0
1,se stejnou upřímností a váhou jako je Pravdivá...,0
2,Fakt blbost. 1. díl byl ucházející a dalo se n...,0
3,Jako mladíkovi uprostřed normalizačních sedmde...,0
4,...aneb proč neudělat ze Tří mušketyrů sci-fi....,0


In [ ]:
df = pd.concat([pos_df,neg_df], sort=False)

## Finetuning

In [ ]:
bs=64
data_lm = (TextList.from_df(df, path, cols='text', processor=SPProcessor.load("."))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))
data_lm.save('cs_clas_databunch')

In [ ]:
data_lm = load_data(path, 'cs_clas_databunch', bs=bs)
data_lm.show_batch()

idx,text
0,"o . ▁a ▁o pet ▁nez klam ala ▁ perfekt ni ▁ muzik a ! ! ! ▁xxbos ▁xxmaj ▁ten hle ▁film ▁ m ů ž u ▁opravdu ▁kdykoliv . ▁a ▁to ▁nejen ▁díky ▁par fek tní mu , ▁ hlá ška mi ▁na bit ému ▁scénář i ▁a ▁herecký m ▁výkonů m ▁všech ▁z ů čas t ně ných ▁( hlavně ▁pak ▁trojice ▁xxmaj ▁troj an , ▁xxmaj ▁mit"
1,". ▁xxbos ▁ špi čkový ▁komerční ▁ válečný ▁seriál , ▁který ▁mě ▁do ▁druhé ▁světové ▁války ▁v táhl ▁snad ▁nejlépe ▁ze ▁všech ▁filmů ▁i ▁seriál ů ▁vůbec . ▁xxmaj ▁osudy ▁dotyčný ch ▁vojáků ▁se ▁nevy poč it atel ně ▁kříží , ▁až ▁to ▁ve ▁ m ne ▁mnohdy ▁vyvolá valo ▁deprese ▁a ▁pocity ▁úzkost i . ▁xxmaj ▁všechny ▁příběhy , ▁zřejmě ▁měly ▁vz bud it ▁v ▁di vá cích ▁odraz"
2,". . ▁xxbos ▁xxmaj ▁za ▁socialistické ho ▁xxmaj ▁československa ▁u ▁nás ▁běžel ▁tento ▁film ▁s ▁názvem ▁"" smrt ▁xxmaj ▁jo a ▁xxmaj ▁indián a "" ▁a ▁jako ▁deseti leté mu ▁kluk ovi ▁se ▁ mi ▁to ▁moc ▁líbil o . ▁xxmaj ▁je ▁to ▁vlastně ▁3. ▁díl ▁čtyř dílné ▁televizní ▁série ▁http : ▁/ ▁/ ▁www . ama zon . de ▁/ ▁ saw y ers - hu c kle berry"
3,"▁mc cl a ne ▁na ▁letišti ▁ve ▁xxmaj ▁washingtonu ▁čeká ▁na ▁svou ▁manželku . ▁xxmaj ▁nezná m ▁lepší ▁způsob ▁jak ▁s tráv it ▁to hle ▁par né ▁letní ▁odpoledne ▁(12 . 7. 200 6, ▁teplota ▁xxup ▁32 ` c ▁ve ▁stínu ) ▁než ▁ve ▁společnosti ▁xxmaj ▁johna ▁a ▁všeho schop ných ▁terorist ů . ▁xxmaj ▁bruce ▁xxmaj ▁will is ▁je ▁jako ▁mc cl a ne ▁prostě ▁bez c hyb ný"
4,"▁s ▁"" to o - big - to - fa il "" ▁banka mi ▁není ▁úplně ▁tak ▁od ▁věci ▁mys let ▁si , ▁že ▁je ▁možné ▁aby ▁nějaký ▁pri vat e ▁ e quit y ▁f und ▁měl ▁svojí ▁velikostí ▁ ambi ci ▁ovládat ▁svět ▁a ▁opravdu ▁tak ▁či nit . ▁xxmaj ▁někomu ▁může ▁v a dit ▁dlouhá ▁stop áž ▁a ▁ne - ma in stream ové ▁režisér ské ▁pojetí"


In [ ]:
lm_fns = ['./cs_wt', './cs_wt_vocab']
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [ ]:
learn_lm.predict("Brno je krásné", n_words=10, temperature=0.1)

'Brno je krásné , ▁ skromn é ▁a ▁ skromn é ▁město ,'

In [ ]:
lr = 1e-3
lr *= bs/48

In [ ]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.536963,4.583334,0.242857,02:47


In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.379922,4.519659,0.271429,03:19
1,4.274517,4.487927,0.228571,03:18
2,4.132161,4.311273,0.257143,03:18
3,3.970825,4.363219,0.242857,03:18
4,3.819635,4.366664,0.257143,03:19


In [ ]:
lang = "cs"
learn_lm.save(f'{lang}_csfd_2classes_fine_tuned')
learn_lm.save_encoder(f'{lang}_csfd_2classes_fine_tuned_enc')

## Classifier

In [ ]:
path_clas = path/'csfd'
bs=64

data_clas = (TextList.from_df(df, path, cols='text', processor=SPProcessor.load("."))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}_csfd_2classes_fine_tuned_enc')
learn_c.freeze()

In [ ]:
lr = 2e-2
lr *= bs/48

In [ ]:
learn_c.fit_one_cycle(3, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.285737,0.230675,0.909916,01:08
1,0.281006,0.229818,0.906616,00:58
2,0.279418,0.217558,0.912391,01:04


In [ ]:
import warnings

learn_c.freeze_to(-2)

with warnings.catch_warnings(record=True):
  learn_c.fit_one_cycle(3, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.237650,0.201547,0.919815,01:20
1,0.206178,0.173994,0.929220,01:12
2,0.168043,0.167931,0.935489,01:11


In [ ]:
learn_c.freeze_to(-3)

with warnings.catch_warnings(record=True):
  learn_c.fit_one_cycle(3, slice(lr/4/(2.6**4),lr/4), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.203157,0.212193,0.916515,01:53
1,0.154562,0.150394,0.939614,02:16
2,0.086916,0.153920,0.945224,02:23


In [ ]:
learn_c.save('/content/drive/My Drive/ULMFit-Czech/cs_csfd_2classes_945')
learn_c.export('/content/drive/My Drive/ULMFit-Czech/cs_csfd_2classes_945.pkl')
learn_c.data.vocab.save('/content/drive/My Drive/ULMFit-Czech/cs_csfd_2classes_945_vocab.pkl')